
# Descriptive information about anesthetists

So far, no one has described the current data available for the number or distribution of the various cadres of anethetists in Uganda. 

My primary questions concern both physician anesthestics and non-physician anesthestists. Primarily,
- How many are there?
- Where are they (ie, distributed rural vs urban)?
- How were they educated?
- How much money do they make?
- How many are being trained?

Note: My citations will look funny in this notebook. Please see this [BibTex file](https://github.com/alexgoodell/uganda-model/blob/master/refs/library.bib) for the current list of my citations.

## Physician anesthetists

According to the WFSA, in Uganda, there are 3,361 physicians (for a physician-to-population ratio of 0.18 per 100,000), 263 of which are surgerons, 72 of which are physican anesthtists, and 47 physician providers that have an anaesthetic qualification. <cite data-cite="wfsaug2018"><a href="#ref-wfsaug2018">(wfsaug2018)</a></cite>. In 2004, 2005, and 2015 (the only data published by the WHO) there were 2209, 3361, and 3645 physicians country-wide. <cite data-cite="whogho2018"><a href="#ref-whogho2018">(whogho2018)</a></cite>

Data concerning the number and location of physician anesthetists was obtained from the Uganda Medical and Dental Practitioners Council Registry website <cite data-cite="umdpc2018reg"><a href="#ref-umdpc2018reg">(umdpc2018reg)</a></cite>.  Extracted data can be found in our GitHub repository, under "data/umdpc/." Filenames were changed to exclude spaces and extract date was included in the title. Data was cleaned by removing whitespace and formatting oddities as documented. 

In [24]:
# import requirements (all found in /model/py/requirements.txt)
import pandas as pd
import numpy as np
import os

# read in data - skip first three rows (header information)
df = pd.read_excel("../../data/umdpc/specialist-register-3-13-2018.xls", skiprows=3, header=0) 
df.head()

,REG NO,SURNAME,OTHER NAME,SEX,EMPLOYER,POSTAL ADDRESS,BOX,CITYTOWN,FIRST QUALIFICATION,PROFES,...,PROFES03,AWARD03,COUNTRY03,YEAR03,Unnamed: 30,PROFES04,AWARD04,COUNTRY04,YEAR04,SPECIALITY
0,ANAESTHESIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,F2820,ATURIA,SARAH AKOL,F,NaN,"P.O BOX 8291, KAMPALA",8291,KAMPALA,"MBChB,MUK,UGANDA,1995",MBChB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANAESTHESIA
2,I3267,AYEBALE,EMMANUEL TIMARWA,M,MAKERERE UNIVERSITY,"P.O BOX 7051, KAMPALA",7051,KAMPALA,"MBChB,MUK,UGANDA,2008",MBChB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANAESTHESIOLOGY
3,J4241,AYUPO,NODREEN,F,UGANDA HEART INSTITUTE,"P.O BOX 7051, KAMPALA",7051,KAMPALA,"MBChB,MUK,UGANDA,2011",MBChB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANAESTHESIA
4,G1128,BUGEMBE,RICHARD NATHAN,M,JINJA HOSPITAL,"P.O BOX 43, JINJA",43,JINJA,"MBChB,MUST,UGANDA,1996",MBChB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANAESTHESIOLOGY


In [25]:
# Fix the whitespace in the column names, replace strings with underscore
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
df.columns

Index([                     u'reg_no',                     u'surname',
                        u'other_name',                         u'sex',
                          u'employer',              u'postal_address',
                               u'box',                    u'citytown',
               u'first_qualification',                      u'profes',
                             u'award',                     u'country',
                              u'year',  u'date_of_first_registration',
       u'additional_qualification(s)',                 u'unnamed:_15',
                          u'profes01',                     u'award01',
                         u'country01',                      u'year01',
                       u'unnamed:_20',                    u'profes02',
                           u'award02',                   u'country02',
                            u'year02',                 u'unnamed:_25',
                          u'profes03',                     u'award03',
      

In [27]:
# Let's find what all these columns are for. Find all rows with no null values in any column
any_null = df.isnull().any(axis=1)
not_any_null = any_null.apply(lambda x: not x)
df[not_any_null][['first_qualification', 'profes', 'award', 'country', 'year', 'additional_qualification(s)', 'unnamed:_15','profes01','award01','country01', 'year01','unnamed:_20', 'profes02','award02', 'country02','year02', 'unnamed:_25','profes03', 'award03','country03', 'year03', 'unnamed:_30', 'profes04','award04', 'country04','year04' ]]

,first_qualification,profes,award,country,year,additional_qualification(s),unnamed:_15,profes01,award01,country01,...,unnamed:_25,profes03,award03,country03,year03,unnamed:_30,profes04,award04,country04,year04
58,"MBChB,MUK,UGANDA,1979",MBChB,MUK,UGANDA,1979,"M MED INTERN MED,MUK,UGANDA,1990,DIP DERMATO,U...","M MED INTERN MED,MUK,UGANDA,1990",M MED INTERN MED,MUK,UGANDA,...,"MSc DERMATO,WALES,UK,1997",MSc DERMATO,WALES,UK,1997,"DIP. HEALTH MGT,ICMI,WALES,2000",DIP. HEALTH MGT,ICMI,WALES,2000
65,"MBChB,EA,UGANDA,1964",MBChB,EA,UGANDA,1964,"FRCP,EDINBURGH,UK,1974,FRCP,LONDON,UK,1979,LMS...","FRCP,EDINBURGH,UK,1974",FRCP,EDINBURGH,UK,...,"LMS,EA,UGANDA,1960",LMS,EA,UGANDA,1960,"MRCP,EDINBURGH,UK,1967",MRCP,EDINBURGH,UK,1967
90,"MBChB,MUK,UGANDA,1978",MBChB,MUK,UGANDA,1978,"PhD,DUBLIN,IRELAND,1987,MRCP,RCP,IRELAND,1987,...","PhD,DUBLIN,IRELAND,1987",PhD,DUBLIN,IRELAND,...,"FDE,MANIPAL,INDIA,2001",FDE,MANIPAL,INDIA,2001,"MSC,GLOBAL HEALTH DUBLIN,UK,2007",MSC,GLOBAL HEALTH DUBLIN,UK,2007
158,"MBChB,MUK,UGANDA,1992",MBChB,MUK,UGANDA,1992,"M.FAM.MED,MEDUNSA,SOUTH AFRICA,2001,DIP.PEC,CM...","M.FAM.MED,MEDUNSA,SOUTH AFRICA,2001",M.FAM.MED,MEDUNSA,SOUTH AFRICA,...,"FCEM,SA,SA,2009",FCEM,SA,SA,2009,"DIP FOR MED,SA,SA,1998",DIP FOR MED,SA,SA,1998
899,"MBChB,MUK,UGANDA,1984",MBChB,MUK,UGANDA,1984,"MPH,WITWATERSRAND,SA,2001,MSc,MUK,UGANDA,1988,...","MPH,WITWATERSRAND,SA,2001",MPH,WITWATERSRAND,SA,...,"DIP. PRIMARY EMERGENCY CARE,WITWATERSRAND,SA,1992",DIP. PRIMARY EMERGENCY CARE,WITWATERSRAND,SA,1992,"DTM & H,WITWATERSRAND,SA,1994",DTM & H,WITWATERSRAND,SA,1994
989,"MBChB,MUK,UGANDA,1987",MBChB,MUK,UGANDA,1987,"DIP COMP Sc,MUK,UGANDA,1989,MPH,LIVERPOOL,UK,1...","DIP COMP Sc,MUK,UGANDA,1989",DIP COMP Sc,MUK,UGANDA,...,"MA DEMO,MUK,UGANDA,1995",MA DEMO,MUK,UGANDA,1995,"PhD,COPENHAGEN,DENMARK,2006",PhD,COPENHAGEN,DENMARK,2006
1010,"MBChB,MUK,UGANDA,1992",MBChB,MUK,UGANDA,1992,"DIP. INFECT. DISEASES,LONDON,UK,2002,MSc,LONDO...","DIP. INFECT. DISEASES,LONDON,UK,2002",DIP. INFECT. DISEASES,LONDON,UK,...,"MPH,WASHINGTON,USA,2013",MPH,WASHINGTON,USA,2013,"PhD,WASHINGTON,USA,2016",PhD,WASHINGTON,USA,2016
1497,"MBChB,MUK,UGANDA,1982",MBChB,MUK,UGANDA,1982,"DTM & H,LONDON,UK,1988,M MED ENT SURG,MUK,UGAN...","DTM & H,LONDON,UK,1988",DTM & H,LONDON,UK,...,"MSc AUDIO,MANCHESTER,UK,1995",MSc AUDIO,MANCHESTER,UK,1995,"DTM & H,LONDON,UK,1988",DTM & H,LONDON,UK,1988
1515,"MBChB,BIRMINGHAM,UK,1985",MBChB,BIRMINGHAM,UK,1985,"FRCS,RCS,UK,1989,FRCS PLASTIC SURG.,RCS,UK,200...","FRCS,RCS,UK,1989",FRCS,RCS,UK,...,"M.MED PATHOLOGY,MUK,UGANDA,2005",M.MED PATHOLOGY,MUK,UGANDA,2005,"DIP. FLORENSIC PATHOLOGY,SA,SA,2012",DIP. FLORENSIC PATHOLOGY,SA,SA,2012


In [28]:
# Rename columns to more useful 
df = df.rename(columns={
	'first_qualification': 'concat_degree2',
	'profes': 'degree_degree2',
	'award': 'program_degree2',
	'country': 'country_degree2',
	'year': 'year_degree2',
	'unnamed:_15': 'concat_degree2',
	'profes01': 'degree_degree2',
	'award01': 'program_degree2',
	'country01': 'country_degree2',
	'year01': 'year_degree2',
	'unnamed:_20': 'concat_degree3',
	'profes02': 'degree_degree3',
	'award02': 'program_degree3',
	'country02': 'country_degree3',
	'year02': 'year_degree3',
	'unnamed:_25': 'concat_degree4',
	'profes03': 'degree_degree4',
	'award03': 'program_degree4',
	'country03': 'country_degree4',
	'year03': 'year_degree4',
	'unnamed:_30': 'concat_degree5',
	'profes04': 'degree_degree5',
	'award04': 'program_degree5',
	'country04': 'country_degree5',
	'year04': 'year_degree5'
})
df.columns

Index([                     u'reg_no',                     u'surname',
                        u'other_name',                         u'sex',
                          u'employer',              u'postal_address',
                               u'box',                    u'citytown',
                    u'concat_degree2',              u'degree_degree2',
                   u'program_degree2',             u'country_degree2',
                      u'year_degree2',  u'date_of_first_registration',
       u'additional_qualification(s)',              u'concat_degree2',
                    u'degree_degree2',             u'program_degree2',
                   u'country_degree2',                u'year_degree2',
                    u'concat_degree3',              u'degree_degree3',
                   u'program_degree3',             u'country_degree3',
                      u'year_degree3',              u'concat_degree4',
                    u'degree_degree4',             u'program_degree4',
      

In [23]:
# They added columns to differentiate specialties. These need to be removed. Let's see if they have any other info
df[df.surname.isnull()][["reg_no","surname", "other_name", "sex"]].head()

# Remove those rows
df = df[df.surname.notnull()]
df.head()

,reg_no,surname,other_name,sex,employer,postal_address,box,citytown,first_qualification,profes,...,degree_degree3,program_degree3,country_degree3,year_degree13,concat_degree4,degree_degree4,program_degree4,country_degree4,year_degree4,speciality
1,F2820,ATURIA,SARAH AKOL,F,NaN,"P.O BOX 8291, KAMPALA",8291,KAMPALA,"MBChB,MUK,UGANDA,1995",MBChB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANAESTHESIA
2,I3267,AYEBALE,EMMANUEL TIMARWA,M,MAKERERE UNIVERSITY,"P.O BOX 7051, KAMPALA",7051,KAMPALA,"MBChB,MUK,UGANDA,2008",MBChB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANAESTHESIOLOGY
3,J4241,AYUPO,NODREEN,F,UGANDA HEART INSTITUTE,"P.O BOX 7051, KAMPALA",7051,KAMPALA,"MBChB,MUK,UGANDA,2011",MBChB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANAESTHESIA
4,G1128,BUGEMBE,RICHARD NATHAN,M,JINJA HOSPITAL,"P.O BOX 43, JINJA",43,JINJA,"MBChB,MUST,UGANDA,1996",MBChB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANAESTHESIOLOGY
5,C0139,BUKWIRWA,HENRY WILLIAM,M,NaN,"P.O BOX 45, KIHIHI",45,KIHIHI,"MBChB,MUK,UGANDA,1980",MBChB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANAESTHESIA


<div id="refs" class="references">
<div id="ref-aau2018" stlye="margin-top:10px">
<p> --- aau2018 --- Association of Anesthesiologist of Uganda. (2018). Bachelors of science in anaesthesia. Retrieved from <a href="http://www.anesthesiaug.org/bachelors/" class="uri">http://www.anesthesiaug.org/bachelors/</a></p>
</div>
<div id="ref-ahpcschools2016">
<p> --- ahpcschools2016 --- uganda allied health professionals council. (2016). AHPC recognized training institutions 2016. Retrieved from <a href="http://www.ahpc.ug/schools.php" class="uri">http://www.ahpc.ug/schools.php</a></p>
</div>
<div id="ref-Banerjee2005Mar">
<p> --- Banerjee2005Mar --- Banerjee, S., Faiz, O., Rennie, J. A., Balyejjusa, J., &amp; Walsh, M. (2005). Bridging the health gap in Uganda: The surgical role of the clinical officer. <em>Afr. Health Sci.</em>, <em>5</em>(1), 86–89. Retrieved from <a href="http://www.ncbi.nlm.nih.gov/pmc/articles/PMC1831898" class="uri">http://www.ncbi.nlm.nih.gov/pmc/articles/PMC1831898</a></p>
</div>
<div id="ref-dsco2012">
<p> --- dsco2012 --- DISTRICT SERVICE COMMISSIONS OF UGANDA. (2012). DISTRICT service commissions of uganda health vacancies. DISTRICT SERVICE COMMISSIONS OF UGANDA. Retrieved from <a href="http://health.go.ug/docs/final_advert.pdf" class="uri">http://health.go.ug/docs/final_advert.pdf</a></p>
</div>
<div id="ref-examboard2015">
<p> --- examboard2015 --- Uganda Allied Health Examinations Board. (2015). Newsletter. Retrieved from <a href="http://uaheb.go.ug/wp-content/uploads/2016/06/Newsletter.pdf" class="uri">http://uaheb.go.ug/wp-content/uploads/2016/06/Newsletter.pdf</a></p>
</div>
<div id="ref-examboardschools">
<p> --- examboardschools --- Uganda Allied Health Examinations Board. (2018). Accredited school. Retrieved from <a href="http://uaheb.go.ug/accredited-schools/" class="uri">http://uaheb.go.ug/accredited-schools/</a></p>
</div>
<div id="ref-hcsurvey2014">
<p> --- hcsurvey2014 --- African Development Bank, M. W. (2014). Uganda hospital and health centre iv census survey. Retrieved from <a href="http://www.who.int/healthinfo/systems/SARA_H_UGA_Results_2014.pdf" class="uri">http://www.who.int/healthinfo/systems/SARA_H_UGA_Results_2014.pdf</a></p>
</div>
<div id="ref-intra2015">
<p> --- intra2015 --- Vincent Oketcho, J. O., Isaac Ezati. (2015). UGANDA on track for keeping human resources for health commitments. Retrieved from <a href="http://www.who.int/workforcealliance/media/news/2015/HRH_Commitments_Uganda_Case_Study.pdf" class="uri">http://www.who.int/workforcealliance/media/news/2015/HRH_Commitments_Uganda_Case_Study.pdf</a></p>
</div>
<div id="ref-kakande2011review">
<p> --- kakande2011review --- Kakande, I., Mkandawire, N., &amp; Thompson, M. (2011). A review of surgical capacity and surgical education programmes in the cosecsa region. <em>East and Central African Journal of Surgery</em>, <em>16</em>(3), 6–34. Retrieved from <a href="http://www.bioline.org.br/request?js11047" class="uri">http://www.bioline.org.br/request?js11047</a></p>
</div>
<div id="ref-Klopper2012-pp">
<p> --- Klopper2012-pp --- Klopper, H., &amp; Uys, L. R. (2012). <em>The State of Nursing and Nursing Education in Africa: A {Country-By-Country} Review</em>. Sigma Theta Tau.</p>
</div>
<div id="ref-lacoranes2018">
<p> --- lacoranes2018 --- Lacor School of Anesthesia. (2018). Lacor school of anesthesia. Retrieved from <a href="http://mambosms.ug/Data/atha/lacorh/?page_id=7346" class="uri">http://mambosms.ug/Data/atha/lacorh/?page_id=7346</a></p>
</div>
<div id="ref-Linden2012-sw">
<p> --- Linden2012-sw --- Linden, A. F., Sekidde, F. S., Galukande, M., Knowlton, L. M., Chackungal, S., &amp; McQueen, K. A. K. (2012). Challenges of surgery in developing countries: a survey of surgical and anesthesia capacity in Uganda’s public hospitals. <em>World J. Surg.</em>, <em>36</em>(5), 1056–1065.</p>
</div>
<div id="ref-mak2018">
<p> --- mak2018 --- Makarere University. (n.d.). Application requirements - direct entry. Retrieved from <a href="https://www.mak.ac.ug/application-procedures/direct-entry" class="uri">https://www.mak.ac.ug/application-procedures/direct-entry</a></p>
</div>
<div id="ref-MOH-hosp-list">
<p> --- MOH-hosp-list --- Ministry of Health. (2018). List of Hospitals. Retrieved from <a href="http://health.go.ug/affiliated-institutions/hospitals" class="uri">http://health.go.ug/affiliated-institutions/hospitals</a></p>
</div>
<div id="ref-mohscholar2017">
<p> --- mohscholar2017 --- Ministry of Health. (2017). ANNOUNCEMENT of scholarships for academic year 2017/2018. Retrieved from <a href="https://www.mnh.musph.ac.ug/wp-content/uploads/2016/10/MIN-OF-HEALTH.-scholarships.pdf" class="uri">https://www.mnh.musph.ac.ug/wp-content/uploads/2016/10/MIN-OF-HEALTH.-scholarships.pdf</a></p>
</div>
<div id="ref-mugalopara2013">
<p> --- mugalopara2013 --- Uganda Allied Health Institute and Management Sciences Mugalo. (2018). Post-basic programs uganda allied health institute and management sciences mugalo. Retrieved from <a href="http://uiahms.ac.ug/anaesthesia/" class="uri">http://uiahms.ac.ug/anaesthesia/</a></p>
</div>
<div id="ref-must2014">
<p> --- must2014 --- Mbarara University of Science and Technology. (n.d.). Admission Requirements. Retrieved from <a href="http://www.must.ac.ug/news/2014-must-private-admissions-advert" class="uri">http://www.must.ac.ug/news/2014-must-private-admissions-advert</a></p>
</div>
<div id="ref-roth2014role">
<p> --- roth2014role --- Roth, R., Frost, E. A., Gevirtz, C., &amp; Atcheson, C. L. (2014). <em>The role of anesthesiology in global health: A comprehensive guide</em>. Springer.</p>
</div>
<div id="ref-UBS2017">
<p> --- UBS2017 --- Ugandan Bureau of Statistics. (2017). Statistical Abstract.</p>
</div>
<div id="ref-umdpc2018reg">
<p> --- umdpc2018reg --- Uganda Medical and Dental Practitioners Council. (n.d.). Uganda medical and dental practitioners council registry. Retrieved from <a href="http://www.umdpc.com/registers.php" class="uri">http://www.umdpc.com/registers.php</a></p>
</div>
<div id="ref-UNFPA2009">
<p> --- UNFPA2009 --- United Nations Fund for Population Activities. (2009). The State of Midwifery Training, Service and Practice in Uganda Assessment Report, (July), 1–92.</p>
</div>
</div>